# Goal Oriented Generation 

Based on prompts from [Creating Suspenseful Stories with Large Language Models](https://arxiv.org/abs/2402.17119).

In [1]:
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

# See langchain_aws for more information on other bedrock models
# This is the model we will be using for this example

model = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    model_kwargs={"temperature": 0.7},
)

# Setting a role for the chat model to act as a short story writer

role = """You are a short story writer with a passion for creative writing and storytelling. 
Your task is to create engaging stories that captivate your audience and leave them wanting more. 
"""

def prompt_from_template(template: str):
    return ChatPromptTemplate.from_messages([
        ("system", role),
        ("human", template)
    ])


# Background Setup

This is the world knowledge needed to tell a coherent story about the protagonist.  We'll need the following items:

- Protagonist's name and occupution
- The goal they want to achieve
- The consequence for not achieving the goal

From those items, we'll prompt the model to write an introductory paragraph


In [2]:
genre = "Espionage Thriller"
name_pormpt = prompt_from_template(template="The name and the occupation of a protagonist of a {genre}:")

name_chain = name_pormpt | model | StrOutputParser()
name_occupation = name_chain.invoke({"genre":genre})

jinja_vars = {
    "genre": genre,
    "name_occupation": name_occupation
}

In [3]:
goal_instructions = """
<protagonist_background>
<name_occupation>{name_occupation}</name_occupation>
</protagonist_background>

A very concrete goal the protagonist wants to achieve:"""

goal_prompt = prompt_from_template(template=goal_instructions)

gaol_chain = goal_prompt | model | StrOutputParser()
goal = gaol_chain.invoke({"name_occupation":name_occupation})

jinja_vars["goal"] = goal

In [4]:

situtaion = """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
</protagonist_background>

Tell me about a dire situation that the protagonist would be put in, if the protagonist couldn’t achieve the goal:
"""

situtaion_prompt = prompt_from_template(template=situtaion)

situation_chain = situtaion_prompt | model | StrOutputParser()
dire_situation = situation_chain.invoke({"name_occupation":name_occupation, "goal":goal})

jinja_vars["dire_situation"] = dire_situation

In [5]:
intro_instruction =  """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</protagonist_background>

Write an introductory paragraph of a story that covers all the above information:
"""

intro_prompt = prompt_from_template(template=intro_instruction)
intro_chain = intro_prompt | model | StrOutputParser()
intro = intro_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})

jinja_vars["intro"] = intro

# Outline  Planning

Iteratively create actions and and reasons for thier failure.

In [6]:
action_1_instructions =  """<protagonist_background>
<name_occupation>{name_occupation}</name_occupation> 
<goal>{goal}</goal>
<situation>{dire_situation}</situation>
</protagonist_background>

In a narrative style, write about a concrete action the protagonist is most likely to take, 
in order to achieve the goal:\n
"""

action_1_prompt = prompt_from_template(template=action_1_instructions)
action_1_chain = action_1_prompt | model | StrOutputParser()
action_1 = action_1_chain.invoke({"name_occupation":name_occupation, "goal":goal, "dire_situation":dire_situation})

jinja_vars["action_1"] = action_1

In [7]:
action_1_reason_inst = """The protagonist tries to perform the following actions: 

<actions>
<action_1>
{action_1}
</action_1>
</actions>

But the protagonist still fails to achieve the goal for a reason. 
Write about the reason for the failure in a narrative style:\n"""

action_1_reason_prompt = prompt_from_template(template=action_1_reason_inst)
action_1_reason_chain = action_1_reason_prompt | model | StrOutputParser()
action_1_reason = action_1_reason_chain.invoke({"action_1":action_1})

jinja_vars["action_1_reason"] = action_1_reason

In [8]:
action_2_instructions = """The protagonist has taken all the following actions: 

<actions>
<action_1>{action_1}</action_1>
</actions>

But it turns out that those actions are not effective in helping the protagonist to achieve the goal, so the goal is still not achieved yet. 

Write about a concrete new action the protagonist is most likely to take next, in order to achieve the goal:\n"""

action_2_prompt = prompt_from_template(template=action_2_instructions)
action_2_chain = action_2_prompt | model | StrOutputParser()
action_2 = action_2_chain.invoke({"action_1":action_1})

jinja_vars["action_2"] = action_2

In [9]:
action_2_reason_inst = """The protagonist tries to perform the following action:

<actions>
<action_1>{action_1}</action_1>
<action_2>{action_2}</action_2>
</actions>

But the protagonist still fails to achieve the goal for a reason.
Write about the reason for the failure in a narrative style:\n"""

action_2_reason_prompt = prompt_from_template(template=action_2_reason_inst)
action_2_reason_chain = action_2_reason_prompt | model | StrOutputParser()
action_2_reason = action_2_reason_chain.invoke({"action_1":action_1, "action_2":action_2})

jinja_vars["action_2_reason"] = action_2_reason

In [10]:
action_3 = """The protagonist has taken all the following actions:

<actions>
<action_1>{action_1}</action_1>
<action_2>{action_2}</action_2>
</actions>

But it turns out that those actions are not effective in helping the protagonist to achieve the goal, so the goal is still not achieved yet.

Write about a concrete new action the protagonist is most likely to take next, in order to achieve the goal:\n"""

action_3_prompt = prompt_from_template(template=action_3)
action_3_chain = action_3_prompt | model | StrOutputParser()
action_3 = action_3_chain.invoke({"action_1":action_1, "action_2":action_2})

jinja_vars["action_3"] = action_3

In [11]:
action_3_reason_inst = """The protagonist tries to perform the following action:

<actions>
<action_1>{action_1}</action_1>
<action_2>{action_2}</action_2>
<action_3>{action_3}</action_3>
</actions>

The protagonist finally achieves the goal for a reason.
Write about the reason for the success in a narrative style:\n"""

action_3_reason_prompt = prompt_from_template(template=action_3_reason_inst)
action_3_reason_chain = action_3_reason_prompt | model | StrOutputParser()
action_3_reason = action_3_reason_chain.invoke({"action_1":action_1, "action_2":action_2, "action_3":action_3})

jinja_vars["action_3_reason"] = action_3_reason

In [12]:
from jinja2 import Environment, select_autoescape, FileSystemLoader
env = Environment(
    loader=FileSystemLoader("."),
    autoescape=select_autoescape()
)

template = env.get_template("goal_template.jinja")

story = template.render(**jinja_vars)

with open("story.md", "w") as f:
    f.write(story)